# Part 1

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extraction = requests.get(url).text
wikiData = BeautifulSoup(extraction, 'lxml')
wikiData;

In [3]:
#Converting to dataframe
columns = ['Postcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = columns)

content = wikiData.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [4]:
#cleaning up
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
        
df = toronto.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.describe()

,Postcode,Borough,Neighborhood
count,180,180,180
unique,180,11,100
top,M1G\n,Not assigned\n,Not assigned\n
freq,1,77,77


In [5]:
#Dropping not assigned values
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [7]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postcode', 'Borough'])
df_2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')
df_2.head()
df_2.drop(0)

,Postcode,Borough,Neighborhood
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn
5,M1H\n,Scarborough\n,Cedarbrae
...,...,...,...
175,M9V\n,Etobicoke\n,"South Steeles, Silverstone, Humbergate, Jamest..."
176,M9W\n,Etobicoke\n,"Northwest, West Humber - Clairville"
177,M9X\n,Not assigned\n,Not assigned\n
178,M9Y\n,Not assigned\n,Not assigned\n


In [11]:
print(df_2.shape)

(180, 3)
